In [1]:
import numpy as np
import scipy.linalg as linalg
from numpy import exp

In [2]:
S0=40; K=40; r=0.1; T=1; sigma=0.3; Smax=100; M=100; N=400; CALLvsPUT='CALL'

In [3]:
dS=Smax/float(M)
dt=T/float(N)
i_values=np.arange(M)
j_values=np.arange(N)
grid=np.zeros(shape=(M+1,N+1))
boundary_conds=np.linspace(0,Smax,M+1)

In [4]:
if CALLvsPUT=='CALL':
    grid[:,-1]= np.maximum(boundary_conds-K,0)
    grid[-1,:-1]= Smax - K*exp(-r*dt*(N-j_values))  
if CALLvsPUT=='PUT':
    grid[:,-1]= np.maximum(K-boundary_conds,0)
    grid[0,:-1]=K*exp(-r*dt*(N-j_values))   

In [5]:
a= 0.5*dt*((r*i_values) - (sigma**2 * i_values**2))
b= 1 + dt*((sigma**2 * i_values**2) + r)
c= -0.5*dt*((r*i_values) + (sigma**2 * i_values**2))

coeffs= np.diag(a[2:M],-1) + np.diag(b[1:M]) + np.diag(c[1:M-1],1)

In [6]:
P, L, U = linalg.lu(coeffs)
aux = np.zeros(M-1)
for j in reversed(range(N)):
    aux[0] = np.dot(-a[1], grid[0, j])
    x1 = linalg.solve(L, grid[1:M, j+1]+aux)
    x2 = linalg.solve(U, x1)
    grid[1:M, j] = x2            

In [7]:
np.interp(S0,boundary_conds,grid[:,0])

6.465461905598071

Code with one additional step (one extra line) necessary for CALL Option:

In [8]:
def ImplicitFDS(S0,K,r,T,sigma,Smax,M,N,CALLvsPUT):
    dS=Smax/float(M)
    dt=T/float(N)
    i_values=np.arange(M)
    j_values=np.arange(N)
    grid=np.zeros(shape=(M+1,N+1))
    boundary_conds=np.linspace(0,Smax,M+1)
    
    if CALLvsPUT=='CALL':
        grid[:,-1]= np.maximum(boundary_conds-K,0)
        grid[-1,:-1]= Smax - K*exp(-r*dt*(N-j_values))  
    if CALLvsPUT=='PUT':
        grid[:,-1]= np.maximum(K-boundary_conds,0)
        grid[0,:-1]=K*exp(-r*dt*(N-j_values))
        
    a= 0.5*dt*((r*i_values) - (sigma**2 * i_values**2))
    b= 1 + dt*((sigma**2 * i_values**2) + r)
    c= -0.5*dt*((r*i_values) + (sigma**2 * i_values**2))
    coeffs= np.diag(a[2:M],-1) + np.diag(b[1:M]) + np.diag(c[1:M-1],1)
    
    P, L, U = linalg.lu(coeffs)
    aux = np.zeros(M-1)
    for j in reversed(range(N)):
        aux[0] = np.dot(-a[1], grid[0, j])
        ''' The additional step: '''
        aux[-1]= np.dot(-c[-1],grid[-1,j])      
        x1 = linalg.solve(L, grid[1:M, j+1]+aux)
        x2 = linalg.solve(U, x1)
        grid[1:M, j] = x2  
    
    return np.interp(S0,boundary_conds,grid[:,0])

In [9]:
print(ImplicitFDS(S0=40, K=40, r=0.1, T=1, sigma=0.3, Smax=100, M=100, N=400, CALLvsPUT='CALL'))

6.6878713497688995


In [10]:
A = np.array([
[2., 1., 1.],
[1., 3., 2.],
[1., 0., 0.]])
B = np.array([4., 5., 6.])
# Perform LU decomposition
LU = linalg.lu_factor(A)
x = linalg.lu_solve(LU, B)
print(x)

[  6.  15. -23.]
